In [1]:
import os

In [2]:
%pwd

'/Users/pateld/Documents/End-to-End-Chest-Cancer-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/pateld/Documents/End-to-End-Chest-Cancer-Classification'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Dharil33/End-to-End-Chest-Cancer-Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Dharil33"
os.environ["MLFLOW_TRACKING_PASSWORD"]="9baa8d49cc304e985e2434668632e3ebdd825fe7"

In [7]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

2023-12-19 22:54:20.967993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [12]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artificats_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/Dharil33/End-to-End-Chest-Cancer-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
   raise e

[2023-12-19 23:13:59,778: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-19 23:13:59,781: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-19 23:13:59,782: INFO: common: created directory at: artificats]
Found 102 images belonging to 2 classes.


2023-12-19 23:14:00.292615: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 7s 963ms/step - loss: 28.6426 - accuracy: 0.4314
[2023-12-19 23:14:07,366: INFO: common: json file saved at: scores.json]


2023/12/19 23:14:10 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-12-19 23:14:11,075: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /var/folders/3_/75pl_m7554j2yd27nf4vhxcr0000gn/T/tmpyh7y919a/model/data/model/assets
[2023-12-19 23:14:11,497: INFO: builder_impl: Assets written to: /var/folders/3_/75pl_m7554j2yd27nf4vhxcr0000gn/T/tmpyh7y919a/model/data/model/assets]


[2023-12-19 23:17:56,141: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-19 23:17:56,144: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-19 23:17:56,145: INFO: common: created directory at: artificats]
Found 102 images belonging to 2 classes.


2023-12-19 23:17:56.631196: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 6s 896ms/step - loss: 28.6426 - accuracy: 0.4314
[2023-12-19 23:18:03,131: INFO: common: json file saved at: scores.json]


2023/12/19 23:18:04 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-12-19 23:18:05,707: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /var/folders/3_/75pl_m7554j2yd27nf4vhxcr0000gn/T/tmp41qeggqe/model/data/model/assets
[2023-12-19 23:18:06,073: INFO: builder_impl: Assets written to: /var/folders/3_/75pl_m7554j2yd27nf4vhxcr0000gn/T/tmp41qeggqe/model/data/model/assets]
[2023-12-19 23:22:37,999: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'timeout('The write operation timed out')': /Dharil33/End-to-End-Chest-Cancer-Classification.mlflow/api/2.0/mlflow-artifacts/artifacts/50630585286448bbb3ffc7accb34e6b4/b10e5042ff574393a475da427e8ca623/artifacts/model/data/model/variables/variables.data-00000-of-00001